In [6]:
from currency_symbols import CurrencySymbols
import string
import re
import os
from tqdm import tqdm
import pandas as pd
from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account

# Define the regex patterns
emoji_pattern = (
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
)
accented_characters = u"\u00C0-\u00FF"  # Latin-1 Supplement (accented characters)
special_symbols = '©®™±≠≤≥∞π∑§¶†•′″‰′′←→↑↓↔↕'

# ISO codes for all currencies 
currencies = [
    'AFN', 'EUR', 'ALL', 'DZD', 'USD', 'EUR', 'AOA', 'XCD', 'XCD', 'ARS', 'AMD', 'AWG', 'AUD', 'EUR', 'AZN',
    'BSD', 'BHD', 'BDT', 'BBD', 'BYN', 'EUR', 'BZD', 'XOF', 'BMD', 'INR', 'BTN', 'BOB', 'BOV', 'USD', 'BAM',
    'BWP', 'NOK', 'BRL', 'USD', 'BND', 'BGN', 'XOF', 'BIF', 'CVE', 'KHR', 'XAF', 'CAD', 'KYD', 'XAF', 'XAF',
    'CLP', 'CLF', 'CNY', 'AUD', 'AUD', 'COP', 'COU', 'KMF', 'CDF', 'XAF', 'NZD', 'CRC', 'XOF', 'EUR', 'CUP',
    'CUC', 'ANG', 'EUR', 'CZK', 'DKK', 'DJF', 'XCD', 'DOP', 'USD', 'EGP', 'SVC', 'USD', 'XAF', 'ERN', 'EUR',
    'SZL', 'ETB', 'EUR', 'FKP', 'DKK', 'FJD', 'EUR', 'EUR', 'EUR', 'XPF', 'EUR', 'XAF', 'GMD', 'GEL', 'EUR',
    'GHS', 'GIP', 'EUR', 'DKK', 'XCD', 'EUR', 'USD', 'GTQ', 'GBP', 'GNF', 'XOF', 'GYD', 'HTG', 'USD', 'AUD',
    'EUR', 'HNL', 'HKD', 'HUF', 'ISK', 'INR', 'IDR', 'XDR', 'IRR', 'IQD', 'EUR', 'GBP', 'ILS', 'EUR', 'JMD',
    'JPY', 'GBP', 'JOD', 'KZT', 'KES', 'AUD', 'KPW', 'KRW', 'KWD', 'KGS', 'LAK', 'EUR', 'LBP', 'LSL', 'ZAR',
    'LRD', 'LYD', 'CHF', 'EUR', 'EUR', 'MOP', 'MKD', 'MGA', 'MWK', 'MYR', 'MVR', 'XOF', 'EUR', 'USD', 'EUR',
    'MRU', 'MUR', 'EUR', 'XUA', 'MXN', 'MXV', 'USD', 'MDL', 'EUR', 'MNT', 'EUR', 'XCD', 'MAD', 'MZN', 'MMK',
    'NAD', 'ZAR', 'AUD', 'NPR', 'EUR', 'XPF', 'NZD', 'NIO', 'XOF', 'NGN', 'NZD', 'AUD', 'USD', 'NOK', 'OMR',
    'PKR', 'USD', 'PAB', 'USD', 'PGK', 'PYG', 'PEN', 'PHP', 'NZD', 'PLN', 'EUR', 'USD', 'QAR', 'EUR', 'RON',
    'RUB', 'RWF', 'EUR', 'SHP', 'XCD', 'XCD', 'EUR', 'EUR', 'XCD', 'WST', 'EUR', 'STN', 'SAR', 'XOF', 'RSD',
    'SCR', 'SLE', 'SGD', 'ANG', 'XSU', 'EUR', 'EUR', 'SBD', 'SOS', 'ZAR', 'SSP', 'EUR', 'LKR', 'SDG', 'SRD',
    'NOK', 'SEK', 'CHF', 'CHE', 'CHW', 'SYP', 'TWD', 'TJS', 'TZS', 'THB', 'USD', 'XOF', 'NZD', 'TOP', 'TTD',
    'TND', 'TRY', 'TMT', 'USD', 'AUD', 'UGX', 'UAH', 'AED', 'GBP', 'USD', 'USD', 'USN', 'UYU', 'UYI', 'UYW',
    'UZS', 'VUV', 'VES', 'VED', 'VND', 'USD', 'USD', 'XPF', 'MAD', 'YER', 'ZMW', 'ZWL', 'ZWG', 'XBA', 'XBB',
    'XBC', 'XBD', 'XTS', 'XXX', 'XAU', 'XPD', 'XPT', 'XAG'
]

currency_symbols = [CurrencySymbols.get_symbol(x) for x in currencies]
currency_symbols = [x for x in currency_symbols if x and not x.isalpha()]  # Filter out None values and alphabetic characters

pattern_string = (
    r'[^a-zA-Z0-9\s' +
    ''.join([re.escape(x) for x in string.punctuation]) +
    ''.join([re.escape(x) for x in currency_symbols]) +
    special_symbols +
    emoji_pattern +
    accented_characters +
    ']'
)

pattern = re.compile(pattern_string, re.UNICODE)

def clean_dataframe(df):
    #df_cleaned = df[~df.apply(lambda row: row.astype(str).str.contains(pattern).any(), axis=1)]
    #df_cleaned['text'] = df_cleaned['text'].str.replace('"', ' ')
    #df_cleaned['text'] = df_cleaned['text'].str.strip()
    #df_cleaned['text'] = df_cleaned['text'].str.rstrip()
    #escaped_chars_pattern = re.compile(r'[' + ''.join([re.escape(c) for c in ['\\', '\t', '\n']]) + ']')
    #df_cleaned['text'] = df_cleaned['text'].str.replace(escaped_chars_pattern, ' ', regex=True)
    #excessive_whitespace_pattern = re.compile(r'\s{2,}')
    #df_cleaned['text'] = df_cleaned['text'].str.replace(excessive_whitespace_pattern, ' ', regex=True)
    #df_cleaned.drop(['date'], axis=1, inplace=True)

    df_cleaned = df
    # Convert BoW column from list to string
    df_cleaned['BoW'] = df_cleaned['BoW'].apply(lambda x: ' '.join(map(str, x)))

    ### later when i load this table as a df in google colab
    ### Convert 'BoW' column to ndarray
    ###df['BoW'] = df['BoW'].apply(lambda x: x.split())  # Split by space to get list of words

    return df_cleaned


def process_and_clean_csv(input_path):
    df = pd.read_csv(input_path, header=0, index_col=False, sep=',', encoding='utf-8', on_bad_lines='skip')
    df_cleaned = clean_dataframe(df)
    return df_cleaned

def load_to_bigquery(df, table_ref):
    job = bigquery_client.load_table_from_dataframe(df, table_ref)
    job.result()
    print(f'Loaded DataFrame into {table_ref.table_id}.')

# Service account key
key_path = '/Users/chkapsalis/Downloads/nlp-project-427710-3e1a48df3dba.json'
credentials = service_account.Credentials.from_service_account_file(key_path)

# Google Cloud project id and dataset information
project_id = 'nlp-project-427710'
dataset_id = 'info_operations_clas'  # Replace with your dataset ID
table_id = 'all_tweets_hashed'  # Replace with your table ID

# Initialization of the BigQuery client
bigquery_client = bigquery.Client(project=project_id, credentials=credentials)
storage_client = storage.Client(project=project_id, credentials=credentials)

# Create the dataset if it does not exist
dataset_ref = bigquery_client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)

try:
    bigquery_client.get_dataset(dataset_ref)  # Make an API request.
    print(f"Dataset {dataset_id} already exists.")
except:
    dataset = bigquery_client.create_dataset(dataset)  # Make an API request.
    print(f"Dataset {dataset_id} created.")

# Define the job configuration for loading data
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("userid", "STRING"),
        bigquery.SchemaField("user_reported_location", "INTEGER"),
        bigquery.SchemaField("user_profile_description", "INTEGER"),
        bigquery.SchemaField("follower_count", "INTEGER"),
        bigquery.SchemaField("BoW", "STRING"),
        bigquery.SchemaField("retweet_ratio", "DECIMAL"),
        bigquery.SchemaField("english_tweet_proportion", "DECIMAL"),
        bigquery.SchemaField("earliest_tweet_time", "INTEGER"),
        bigquery.SchemaField("latest_tweet_time", "INTEGER"),
        bigquery.SchemaField("average_tweet_time", "INTEGER"),
        bigquery.SchemaField("median_tweet_time", "INTEGER"),
        bigquery.SchemaField("tweet_count", "INTEGER"),
        bigquery.SchemaField("stddev_tweet_time", "INTEGER"),
        bigquery.SchemaField("mode_0", "INTEGER"),
        bigquery.SchemaField("mode_1", "INTEGER"),
        bigquery.SchemaField("mode_2", "INTEGER"),
        bigquery.SchemaField("mode_3", "INTEGER"),
        bigquery.SchemaField("mode_4", "INTEGER"),
        bigquery.SchemaField("mode_5", "INTEGER"),
        bigquery.SchemaField("mode_6", "INTEGER"),
        bigquery.SchemaField("mode_7", "INTEGER"),
        bigquery.SchemaField("mode_8", "INTEGER"),
        bigquery.SchemaField("mode_9", "INTEGER"),
        bigquery.SchemaField("mode_10", "INTEGER"),
        bigquery.SchemaField("mode_11", "INTEGER"),
        bigquery.SchemaField("mode_12", "INTEGER"),
        bigquery.SchemaField("mode_13", "INTEGER"),
        bigquery.SchemaField("mode_14", "INTEGER"),
        bigquery.SchemaField("mode_15", "INTEGER"),
        bigquery.SchemaField("mode_16", "INTEGER"),
        bigquery.SchemaField("mode_17", "INTEGER"),
        bigquery.SchemaField("mode_18", "INTEGER"),
        bigquery.SchemaField("mode_19", "INTEGER"),
        bigquery.SchemaField("mode_20", "INTEGER"),
        bigquery.SchemaField("mode_21", "INTEGER"),
        bigquery.SchemaField("mode_22", "INTEGER"),
        bigquery.SchemaField("mode_23", "INTEGER"),
        bigquery.SchemaField("avg_tweets_per_week", "DECIMAL"),
        bigquery.SchemaField("avg_tweets_per_day", "DECIMAL"),
        bigquery.SchemaField("avg_tweets_per_hour", "DECIMAL"),
        bigquery.SchemaField("avg_tweets_per_min", "DECIMAL"),
        bigquery.SchemaField("avg_quote_count", "DECIMAL"),
        bigquery.SchemaField("avg_like_count", "DECIMAL"),
        bigquery.SchemaField("avg_retweet_count", "DECIMAL"),
        bigquery.SchemaField("avg_hashtags", "DECIMAL"),
        bigquery.SchemaField("avg_urls", "DECIMAL"),
        bigquery.SchemaField("avg_user_mentions", "DECIMAL"),
        bigquery.SchemaField("info_op", "INTEGER")
    ],
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    field_delimiter='|',  # used this custom delimiter to help make sense out of the data
    autodetect=False,  # Automatically detect the schema
    max_bad_records=2000,  # Allow up to 2000 bad records
    ignore_unknown_values=True  # Ignore unknown values
)

source_folder = '/Users/chkapsalis/Downloads/info_operations_clas'
filename = 'all_tweets_hashed.csv'
source_file = os.path.join(source_folder, filename)
df_cleaned = process_and_clean_csv(source_file)
# making sure that the dtypes of columns match what asserted in the schema
for col in df_cleaned.columns.difference(['userid','BoW']):
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
        
# Upload the cleaned DataFrame to BigQuery
#load_to_bigquery(df_cleaned, dataset_ref.table(table_id))


Dataset info_operations_clas already exists.


In [7]:
df_cleaned.shape

(22873094, 12)

In [8]:
df_cleaned['info_op'].value_counts()

info_op
1    12023624
0    10849470
Name: count, dtype: int64

In [11]:
# Since we have that much data, we can randomly select a balanced sample 
# number of data points to keep from each category
n_samples = 1500000

# Select `n_samples` from each category
df_sampled_1 = df_cleaned[df_cleaned['info_op'] == 1].sample(n=n_samples, random_state=1)
df_sampled_0 = df_cleaned[df_cleaned['info_op'] == 0].sample(n=n_samples, random_state=1)

# Combine the samples into a new DataFrame
df_sampled = pd.concat([df_sampled_1, df_sampled_0])

# sanity check
print(df_sampled['info_op'].value_counts())

# Print the shape of the new DataFrame
print(df_sampled.shape)


info_op
1    1500000
0    1500000
Name: count, dtype: int64
(3000000, 12)


In [9]:
### if it has labels on both sides, upload... else examine...

In [12]:
load_to_bigquery(df_sampled, dataset_ref.table(table_id))


Loaded DataFrame into all_tweets_hashed.


In [14]:
df_sampled.columns

Index(['tweetid', 'userid', 'tweet_time', 'is_retweet', 'quote_count',
       'like_count', 'retweet_count', 'hashtags', 'urls', 'user_mentions',
       'BoW', 'info_op'],
      dtype='object')

In [15]:
df_sampled.head()

,tweetid,userid,tweet_time,is_retweet,quote_count,like_count,retweet_count,hashtags,urls,user_mentions,BoW,info_op
16267258,NaN,2b1e791d8e8d10517899e6cc3d054bf4a4245dff1f1167...,NaN,0,0.0,0.0,0.0,1,0,0,"[ ' l o c a l ' , ' b o d i e s ' , ' i n ...",1
20803182,NaN,33c334fbf5f157f2428ea361a80b9ff2917b813dd09db0...,NaN,0,0.0,0.0,0.0,1,1,0,[ ],1
15781546,NaN,17272bd5ecded41768a166c81ead83238de6481a64652e...,NaN,1,0.0,0.0,0.0,5,1,1,[ ],1
18555705,NaN,9b6ede19a1a2ec1bf0a6914034f9baec6987979196654e...,NaN,0,0.0,0.0,0.0,0,1,0,[ ],1
10865216,NaN,b67ec4ea371ad5854b702c48a6ea20b6c4345b5c1bbc4b...,NaN,0,0.0,0.0,0.0,1,0,4,"[ ' m a t e r i a l ' , ' l o o k ' , ' e ...",1


In [16]:
df_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000000 entries, 16267258 to 8203036
Data columns (total 12 columns):
 #   Column         Dtype  
---  ------         -----  
 0   tweetid        float64
 1   userid         object 
 2   tweet_time     float64
 3   is_retweet     int64  
 4   quote_count    float64
 5   like_count     float64
 6   retweet_count  float64
 7   hashtags       int64  
 8   urls           int64  
 9   user_mentions  int64  
 10  BoW            object 
 11  info_op        int64  
dtypes: float64(5), int64(5), object(2)
memory usage: 297.5+ MB
